<a href="https://colab.research.google.com/github/jamglez/ML-FreeCodeCamp/blob/main/Copia_de_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called predict_message that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the SMS Spam Collection dataset. The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
test_data = pd.read_csv(test_file_path, sep="\t", header=None, names=['class', 'text'])
train_data = pd.read_csv(train_file_path, sep="\t", header=None, names=['class', 'text'])
train_data.head()

In [ ]:
(train_data.shape,test_data.shape)

In [ ]:
data = train_data.append(test_data, ignore_index=True)
data

In [ ]:
classes = data["class"]
texts = data["text"]

label_encoder = LabelEncoder()
classes = label_encoder.fit_transform(classes)
tokenizer = keras.preprocessing.text.Tokenizer(num_words=1000)
tokenizer.fit_on_texts(texts)

mat_data = tokenizer.texts_to_matrix(texts,mode='count')
mat_train = tokenizer.texts_to_matrix(train_data,mode='count')
mat_test = tokenizer.texts_to_matrix(test_data,mode='count')


In [ ]:
x_train = tokenizer.texts_to_sequences(train_data["text"])
x_test = tokenizer.texts_to_sequences(test_data["text"])
train = keras.preprocessing.sequence.pad_sequences(x_train,maxlen=100)
test = keras.preprocessing.sequence.pad_sequences(x_test,maxlen=100)

labeltrain = train_data["class"]
traintags = label_encoder.fit_transform(labeltrain)
labeltest = test_data["class"]
testtags = label_encoder.fit_transform(labeltest)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Embedding(5000, 128))
model.add(keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2,activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc',keras.metrics.binary_accuracy])
model.summary()

In [ ]:
model.fit(train,traintags,batch_size=32,epochs=10,verbose=1,validation_split=0.3)

In [ ]:
_,accuracy,_ =model.evaluate(test,testtags)
accuracy

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  inputstr = tokenizer.texts_to_sequences([pred_text])
  is_spam = model.predict(np.array(inputstr))[0][0]

  if is_spam >= 0.5:
    label = "spam"
  else:
    label = "ham"

  output = [is_spam, label]

  return output

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
